In [1]:
import glob

In [12]:
V_dirs = glob.glob("minimise_results/V_*")

for V_dir in V_dirs:
    V = float(V_dir.split("/")[-1].split("_")[-1])
    mu_dirs = glob.glob(f'{V_dir}/mu_*')
    
    for mu_dir in mu_dirs:
        mu = float(mu_dir.split("/")[-1].split("_")[-1])
        
        U_files = glob.glob(f'{mu_dir}/U_*.hdf5')
        
        for U_file in U_files:
            
    
#fig, axs = plt.subplots(nrows=4*len(V_list), ncols=len(mu_files_test), figsize=(len(mu_files_test)*5, 4*5*len(V_list)))


-1.0
0.0
-2.0
-1.0
0.0
-0.5
